This notebook walks through the process of setting up Kubernetes on a network of one or more Raspberry Pis. It follows a tutorial on the subject titled "[Will it cluster?](https://blog.alexellis.io/test-drive-k3s-on-raspberry-pi/)", with my own annotated notes on things as I go along.

This notebook assumes you've already gone through the initial step of connecting your Raspberry Pi to your router and configuring it for SSH access.

----

## Making the Raspberry Pi discoverable 

### Methods of discovering and associating IP addresses
Any device which connects to a router will by default be assigned a private network IP address by the router using a process known as DHCP negotiation. This IP address comes from one of two network ranges that are allocated to routers for local network communication (see also [ResidentMario/browser-networking-notes](https://github.com/ResidentMario/browser-networking-notes)).

The exact IP addresses allocated and the scheme used to allocate them will vary from router to router. The three ranges are:

* 192.168.0.0 - 192.168.255.255 (65,536 IP addresses)
* 172.16.0.0 - 172.31.255.255 (1,048,576 IP addresses)
* 10.0.0.0 - 10.255.255.255 (16,777,216 IP addresses)

My router, a NETGEAR R6220, assigns 192.168.1.1 to itself, then assigns IP addresses in ascending order of the 192.168.1.* scheme to all other connected devices. This is a pretty typical arrangement. This is known as dynamic IP address assignment.

Dynamic IP address assignment has the problem that the IP address of a device is dependent on the time at which it connected to the network.

Assuming our Pis are connected to the router using dynamic IP addresses, how would we discover them from e.g. my personal machine? This turns to be non-trivial. Routers maintain a list of connected machines, but do not broadcast that information between machines.

If another device tries to communicate with your device over the router network, its IP address to MAC address mapping is added to your machine's in-memory [ARP cache](https://en.wikipedia.org/wiki/ARP_cache). You can use this to figure out what machine you are talking to if you talked to that machine in the past, but that doesn't solve our problem really, it just creates a new chicken-and-egg problem.

Another way of approaching this would to scan every network address in the subdomain using the `nmap` tool. This is an intensive approach.

Another way is assigning the Pi machines static IP addresses. This is considered poor practice from a sysadmin perspective, however.

Another way (and the method that I went with) is to assign the Pi machines consequitive hostnames. Assuming the Raspberry Pi machines have static hostnames, service discovery is then as simple as `ping raspberrypi` on Linux, or `ping raspberrypi.local` on macOS. This requires assigning static hostnames on the machine. See the next section to see how this is done.

References for this section:
* https://raspberrypi.stackexchange.com/questions/12440/ssh-into-raspberry-pi-without-knowing-ip-address
* https://raspberrypi.stackexchange.com/questions/13936/find-raspberry-pi-address-on-local-network

### Assigning static hostname to a Raspberry Pi

The hostname that a device takes on is managed on the device level by the device's DHCP client. This client is managed via the `/etc/dhcp/dhclient.conf` configuration file. The default value is:

    send host-name = gethostname();
    
`gethostname` instructs the client to get the hostname of the device by calling the `gethostname` system call. This system call in turn manages the hostname at `/etc/hostname`, which you can edit to change its value. The value set here will be sent as a DNS record to the network link at startup time.

But editing this file alone is not sufficient to change the hostname. You also need to edit `etc/hosts`. `etc/hosts` is a file containing a list of IP address to hostname mappings that the computer will use before it falls back to using DNS. This file probably contains a `127.0.0.1` to `OLD_HOSTNAME` mapping, which must also be updated.

So to assign a new static hostname:

1. `sudo nano /etc/hostname`, replacing the value there with the new value.
2. `sudo nano /etc/hosts`, replacing the corresponding value there with the new value.
3. `sudo reboot`.

Then run `hostname` to check that everything is kosher.

References for this section:
* https://www.cyberciti.biz/faq/ubuntu-change-hostname-command/
* http://man7.org/linux/man-pages/man2/gethostname.2.html

### Removing the stale DNS record for the old Raspberry Pi name 

Updating the hostname this way does not remove the stale DNS record for `raspberrypi` (on Linux) or `raspberrypi.local` (on macOS) from your local machine. You should delete this record manually yourself, as it is now incorrect.

Use `nsupdate` to do this:

    ```
    nsupdate
    > update delete raspberrypi.local A
    ```

Then run `ping rasbperrypi.local` and make sure this returns a "cannot find".

The DNS system will not remove this old entry itself until the cache time expires, as a mechanism for doing so would be vulnerable in the case of transient failures. So it's best practice to remove this record yourself.

References for this section:
* https://linux.die.net/man/8/nsupdate

### Creating a PTR record for the new Raspberry Pi name [TODO]

RESOLVE WHY THE RASPBERRY PI IS REACHABLE VIA IP BUT NOT VIA HOSTNAME.

Is this necessary? Resolve https://raspberrypi.stackexchange.com/questions/7640/raspberry-pi-not-reachable-via-its-hostname-in-lan, then see https://support.opendns.com/hc/en-us/articles/227988467-What-is-in-addr-arpa-.

### Enabling cgroups features on the Raspberry Pi

`cgroups` is the name of the Linux kernel feature that is used to provide kernel-level resource isolation in Linux. It is optional for Docker, but mandatory for Kubernetes, as it is the kernel feature that Kubernetes uses to provide resource isolation within and between pods.

The RASPIAN bootloader does not enable `cgroups` by default, so we must do so ourselves. On most Linux machines this is done by managing BIOS settings, but the Raspberry Pi does not have a BIOS. Instead, you edit a file on the machine directly, and boot options are read out of that file.

The specific line to be added is:

    ```
    cgroup_enable=cpuset cgroup_memory=1 cgroup_enable=memory
    ```

Then reboot the machine again.

Unfortunately, I was immediately able to find out what these options do, exactly. Searching this string on Google doesn't return much joy.

After rebooting, verify that `cgroups` features have been enabled on your machine by running `cat /proc/mounts | grep /sys/fs/cgroup/`. This should return a bunch of `cgroup` mounts on your machine.

References for this section:
* https://unix.stackexchange.com/questions/427327/how-can-i-check-if-cgroups-are-available-on-my-linux-host

### Install k3s

`k3s` is a lightweight Kubernetes system management layer suitable for use with the Raspberry Pi. It is provided by the folks at Rancher. Install it using its following bootstrap script:

    ```
    curl -sfL https://get.k3s.io | sh -s - --write-kubeconfig-mode 644
    ```

To check that it launched successfully, and that it's been added to the `systemd` boot tasks as desired, run:

    ```
    sudo systemctl status k3s
    ```

Note that the instructions here on installation are different from those presented in the article used for these notes. The instructions have been modified to account for a patch change in the install script. Previously the configuration file `/etc/rancher/k3s/k3s.yaml` was world-readable, which security people didn't like. It has since been updated to give permissions only to a `k3s` permissions group by default.

There are a couple of ways to get back runability without superuser permissions. One is to overwrite the config file permissions to `644`, which is what we do here:

> 644 means you can read and write the file or directory and other users can only read it. Suitable for public text files. ([source](http://www.statslab.cam.ac.uk/~eva/unixinfo/perms.html))

The other way would be to follow the instructions on the [GH#389](https://github.com/rancher/k3s/issues/389#issuecomment-486332171) and `usermod` to add the `admin` group to the permissions group:

> I think the best approach would probably be to create a k3s group and prompt the user to run usermod like docker installation does.

This isn't really all that advantageous, so I'll just go with the 644 option.

If all is well, the following should return output without complaining:

    kubectl get node -o wide

Important note: the runtime environment is raw `containerd`, not `Docker`, which helps save on runtime weight apparently.


### Test working status

To test that everything is working, try deploying a microservice. For this, just follow the instructions on the original blog post, under the section "Deploy a microservice", without modification.